In [2]:
import xarray as xr
import matplotlib.pyplot as plt
# from snakemake.io import 
import pandas as pd
import numpy as np
from thesis_toolbox.utils import get_locations_CLP
import matplotlib.transforms as mtransforms
from matplotlib.ticker import LogLocator
%matplotlib inline

In [3]:
if 'snakemake' in globals():
    to_file = True

else:
    path = '../../../'
    to_file= False
    psize='2micron'
    kind='drydep'

In [4]:
if 'snakemake' in globals():
    to_file = True
    path='./'
#     psize= snakemake.wildcards.psize
#     kind = snakemake.wildcards.kind
else:
    path = '../../../'
    to_file= False
    psize='2micron'
    kind='drydep'

def read_data(paths, locs):
    pdict = {}
    for location in locs:
        tempdfs = [pd.read_csv(p) for p in paths if location == p.split('.')[-5]]
        tempdfs = pd.concat(tempdfs)
        tempdfs.index = pd.to_datetime(tempdfs.time,yearfirst=True)
        tempdfs = tempdfs.drop(columns='time')
        
        pdict[location] = tempdfs.rename(columns={'0':location})
    outdf = pd.DataFrame(columns=locs, index=tempdfs.index)
    for location in locs:
        outdf.loc[:,location] = pdict[location][location].copy()
#     df = pd.concat(dfs)
#     df.index = pd.to_datetime(df.time,yearfirst=True)
#     df = df.drop(columns='time')
    return outdf


def plot_deposition_histogram(df, axes):


    df_early_spring = []
    for year in range(1999,2019):
        df_early_spring.append(df[f'{year}-03-01':f'{year}-04-17'])

    df_late_spring = []
    for year in range(1999,2019):
        df_late_spring.append(df[f'{year}-04-17':f'{year}-05-31'])
    df_early_spring = pd.concat(df_early_spring)
    df_late_spring = pd.concat(df_late_spring)
    logbins = np.logspace(np.log10(1e-5),np.log10(0.1),10)
    for site, ax in zip(df.columns, axes): 
        
        df_late_spring[[site]].where(df_late_spring>1e-5).plot.hist(bins=logbins,ax=ax, 
                                                            label='Late spring',  legend=False,
                                                            alpha=0.7, grid=False, color="#f89c20")
        df_early_spring[[site]].where(df_early_spring>1e-5).plot.hist(bins=logbins,ax=ax,
                                                              label='Early spring',   legend=False,
                                                         alpha=0.7, color= "#5790fc")

        ax.semilogx()
    
"#5790fc", "#f89c20",

In [5]:
locs =  ['SHAPOTOU','SACOL','BAODE','LUOCHUAN','LINGTAI','LANTIAN']
dfdp2m = read_data(snakemake.input.drypdep_2micron, locs)
dfdp20m = read_data(snakemake.input.drydep_20micron, locs)
dfwp2m = read_data(snakemake.input.wetdep_2micron, locs)
dfwp20m = read_data(snakemake.input.wetdep_20micron,locs)

In [6]:
def remove_spines(axes, ylim, side):
    for ax in axes.ravel():
        ax.spines['top'].set_visible(False)
        ax.spines[side].set_visible(False)
        ax.set_ylim(ylim)
        if side =='left':
            ax.yaxis.tick_right()
            ax.yaxis.set_label_position("right")
#         ax.spines['bottom'].set_visible(False)
#         ax.spines['left'].set_visible(False)

In [7]:
def remove_ticks(axes):
    for ax in axes.ravel():
        ax.set_yticklabels([])
        ax.set_ylabel('')

In [8]:
fig,axes = plt.subplots(ncols=4,nrows=6, figsize=(7,8), sharex=True, sharey=False)
plot_deposition_histogram(dfdp2m,axes[:,0])
plot_deposition_histogram(dfdp20m,axes[:,1])
plot_deposition_histogram(dfwp2m,axes[:,2])
plot_deposition_histogram(dfwp20m,axes[:,3])

axes[0,0].set_title('Clay',loc='center',y=.97)
axes[0,1].set_title('Silt',loc='center',y=.97)
axes[0,2].set_title('Clay',loc='center',y=.97)
axes[0,3].set_title('Silt',loc='center',y=.97)
remove_spines(axes[:,:2], ylim=(0,400),side='right')
remove_ticks(axes[:,1:2])
remove_spines(axes[:,2:],ylim=(0,150), side='left')
remove_ticks(axes[:,2:3])
axes[-1,0].set_xticks([1e-5,1e-3,1e-1])
axes[-1,0].set_xticks([1e-4,1e-2], minor=True, labels=[])
# axes[-1,0].xaxis.set_minor_locator(LogLocator(num))
fig.text(0.32,0.94,'Dry-deposition', va='center', ha='center', weight='semibold'
         ,size='large',transform=fig.transFigure)
fig.text(0.71,0.94,'Wet-deposition', va='center', ha='center', weight='semibold'
         ,size='large',transform=fig.transFigure)


fig.text(0.51, 0.07, "Dust deposited per day [g m-2]",va='center', ha='center'
         ,size='large',transform=fig.transFigure)
h,l =axes[0,0].get_legend_handles_labels()

for ax, site in zip(axes[:,0].ravel(), locs):
    ax.set_ylabel(f"{site} \n # events")
for ax in axes[:,-1].ravel():
    ax.set_ylabel('')
    
fig.legend(h, ['Late spring', 'Early spring'],bbox_to_anchor=(0.55,0.055),ncol=2, frameon=False,fontsize='large')
if to_file:
    plt.savefig(snakemake.output.outpath, bbox_inches='tight')